In [1]:
# !pip install ipynbname
import ipynbname
from feupy.utils.tests import get_names_tests
from feupy.analysis import config

/home/born-again/anaconda3/envs/gammapy-1.1/lib/python3.9/site-packages/pydantic/_migration.py:290: UserWarning: `pydantic.utils:deep_update` has been removed. We are importing from `pydantic.v1.utils:deep_update` instead.See the migration guide for more details: https://docs.pydantic.dev/latest/migration/
  warnings.warn(


[NbConvertApp] Converting notebook test_core.ipynb to script
[NbConvertApp] Writing 28493 bytes to core.py


/home/born-again/anaconda3/envs/gammapy-1.1/lib/python3.9/site-packages/pydantic/_migration.py:290: UserWarning: `pydantic.utils:deep_update` has been removed. We are importing from `pydantic.v1.utils:deep_update` instead.See the migration guide for more details: https://docs.pydantic.dev/latest/migration/
  warnings.warn(


In [2]:
script_name, output_dir = get_names_tests(mod_file=config.__file__)

Python script name: config.py
Output directory: /home/born-again/Documents/GitHub/feupy/feupy/analysis/config.py


In [3]:
# from .config import *
from feupy.roi import ROI
from feupy.target import Target
from astropy import units as u
from feupy.utils.scripts import pickling, unpickling
from feupy.plotters import setting_leg_style, show_SED
from feupy.analysis.config import SimulationConfig
from feupy.analysis.core import Simulation
from feupy.cta.irfs import Irfs
from feupy.catalog import CATALOG_REGISTRY

from feupy.utils.string_handling import name_to_txt

from gammapy.utils.scripts import make_path

from gammapy.modeling.models import (
    PowerLawSpectralModel,
    ExpCutoffPowerLawSpectralModel,
    GaussianSpatialModel,
    SkyModel,
    Models,
    FoVBackgroundModel,
)

PULSARS = ['PSR J1826-1334', 'PSR J1826-1256', 'PSR J1837-0604', 'PSR J1838-0537']
PEVATRONS = ['LHAASO J1825-1326', 'LHAASO J1839-0545']
PWNS = ["HESS J1825-137", 'HESS J1826-130', 'HESS J1837-069', 'HESS J1841-055']

catalog_lhaaso = CATALOG_REGISTRY.get_cls('publish-nature-lhaaso')()
catalog_pulsar = CATALOG_REGISTRY.get_cls('ATNF')()
catalog_gammacat = CATALOG_REGISTRY.get_cls('gamma-cat')()
catalog_hees = CATALOG_REGISTRY.get_cls('hgps')()

config = SimulationConfig()
analysis = Simulation(config)
pulsar_name = PULSARS[1]
target_name = pulsar_name 
catalog = catalog_pulsar

target_name = pulsar_name 
catalog = catalog_pulsar
source = catalog[target_name]
name = source.name
pos_ra = source.position.ra
pos_dec = source.position.dec
target = Target(name, pos_ra, pos_dec)

Setting logging config: {'level': 'INFO', 'filename': None, 'filemode': None, 'format': None, 'datefmt': None}


In [4]:
position = target.position
on_region_radius=0.15*u.deg
on_region = {'lon':276.53554167*u.deg,
 'lat': -12.9425 *u.deg,
 'frame': 'icrs',
 'radius': on_region_radius,
}
pointing_angle = 0*u.deg
offset=1*u.deg 

roi = ROI(target=target, radius=on_region_radius)

config.datasets.on_region = on_region
config.observations.pointing.angle = pointing_angle
config.observations.offset = offset

e_edges_min=0.1*u.TeV 
e_edges_max=100.*u.TeV
nbin_edges = 5
name = 'energy'

config.datasets.geom.axes.energy.min = e_edges_min
config.datasets.geom.axes.energy.max = e_edges_max
config.datasets.geom.axes.energy.nbins = nbin_edges
config.datasets.geom.axes.energy.name = 'energy'


ValueError: "ObservationsConfig" object has no field "offset"

In [ ]:
class GammapyBaseConfig(BaseModel):
    class Config:
        validate_all = True
        validate_assignment = True
        arbitrary_types_allowed=True
        from_attributes=True
        extra = "forbid"
        json_encoders = {
            Angle: lambda v: f"{v.value} {v.unit}",
            Quantity: lambda v: f"{v.value} {v.unit}",
            Time: lambda v: f"{v.value}",
        }

        
class SkyCoordConfig(GammapyBaseConfig):
    frame: FrameEnum = None
    lon: AngleType = None
    lat: AngleType = None

        
class EnergyAxisConfig(GammapyBaseConfig):
    min: EnergyType = None
    max: EnergyType = None
    nbins: int = None
    name: str = "energy"


class SpatialCircleConfig(GammapyBaseConfig):
    frame: FrameEnum = None
    lon: AngleType = None
    lat: AngleType = None
    radius: AngleType = None


class EnergyRangeConfig(GammapyBaseConfig):
    min: EnergyType = None
    max: EnergyType = None


class TimeRangeConfig(GammapyBaseConfig):
    start: TimeType = None
    stop: TimeType = None


class FluxPointsConfig(GammapyBaseConfig):
    energy: EnergyAxisConfig = EnergyAxisConfig()
    source: str = "source"
    parameters: dict = {"selection_optional": "all"}


class LightCurveConfig(GammapyBaseConfig):
    time_intervals: TimeRangeConfig = TimeRangeConfig()
    energy_edges: EnergyAxisConfig = EnergyAxisConfig()
    source: str = "source"
    parameters: dict = {"selection_optional": "all"}


class FitConfig(GammapyBaseConfig):
    fit_range: EnergyRangeConfig = EnergyRangeConfig()


class ExcessMapConfig(GammapyBaseConfig):
    correlation_radius: AngleType = "0.1 deg"
    parameters: dict = {}
    energy_edges: EnergyAxisConfig = EnergyAxisConfig()


class BackgroundConfig(GammapyBaseConfig):
    method: BackgroundMethodEnum = None
    exclusion: Path = None
    parameters: dict = {}


class SafeMaskConfig(GammapyBaseConfig):
    methods: List[SafeMaskMethodsEnum] = [SafeMaskMethodsEnum.aeff_default]
    parameters: dict = {}


class EnergyAxisTrueConfig(GammapyBaseConfig):
    min: EnergyType = None
    max: EnergyType = None
    nbins: int = None
    name: str = "energy_true"
        

class EnergyAxesConfig(GammapyBaseConfig):
    energy: EnergyAxisConfig = EnergyAxisConfig()
    energy_true: EnergyAxisTrueConfig = EnergyAxisTrueConfig()


class SelectionConfig(GammapyBaseConfig):
    offset_max: AngleType = "2.5 deg"


class WidthConfig(GammapyBaseConfig):
    width: AngleType = "5 deg"
    height: AngleType = "5 deg"


class WcsConfig(GammapyBaseConfig):
    skydir: SkyCoordConfig = SkyCoordConfig()
    binsize: AngleType = "0.02 deg"
    width: WidthConfig = WidthConfig()
    binsize_irf: AngleType = "0.2 deg"


class GeomConfig(GammapyBaseConfig):
#     wcs: WcsConfig = WcsConfig()
#     selection: SelectionConfig = SelectionConfig()
    axes: EnergyAxesConfig = EnergyAxesConfig()


class DatasetsConfig(GammapyBaseConfig):
    type: ReductionTypeEnum = ReductionTypeEnum.spectrum
#     stack: bool = True
    geom: GeomConfig = GeomConfig()
    selection: List[MapSelectionEnum] = MapDatasetMaker.available_selection
    use_region_center: bool = False
#     background: BackgroundConfig = BackgroundConfig()
#     on_region: SpatialCircleConfig = SpatialCircleConfig()
    containment_correction: bool = False
    safe_mask: SafeMaskConfig = SafeMaskConfig()
        

class DatasetsOnOffConfig(GammapyBaseConfig):
    acceptance: int = None
    acceptance_off: int = None
    
        
        
class SensitivityConfig(GammapyBaseConfig):
    gamma_min: int = None
    n_sigma: int = None
    bkg_syst_fraction: float = None
    table: dict = {}

# class WStatisticsConfig(GammapyBaseConfig):
#     excess: float = None
#     ts: float = None
#     sqrt_ts: float = None

class StatisticsConfig(GammapyBaseConfig):
    alpha: float = None
    wstat: dict = {}
    fitted_parameters: dict = {}   
                
class ObservationsParametersConfig(GammapyBaseConfig):
    livetime: QuantityType = None
    offset: QuantityType = None
    on_region_radius: AngleType = None
    n_obs: int = None


class TargetConfig(GammapyBaseConfig):
    name: str = None
    position: SkyCoordConfig = SkyCoordConfig()
    model: dict = {}  
    model_fitted: dict = {}
        
class IrfsConfig(GammapyBaseConfig):
    opt: IrfType = ['South', 'AverageAz', '20deg', '50h']

        
class PointingConfig(GammapyBaseConfig):
    angle: AngleType = None
        
class ObservationsConfig(GammapyBaseConfig):
    target: TargetConfig = TargetConfig()
    parameters: ObservationsParametersConfig = ObservationsParametersConfig()
    irfs: IrfsConfig = IrfsConfig()
    pointing: PointingConfig = PointingConfig()
        
class ROIConfig(GammapyBaseConfig):
    target: TargetConfig = TargetConfig()
    radius: AngleType = None
    catalogs: CatalogsTypeEnum = "all"
    dict_sep: dict = {} 
    leg_style: dict = {}
        
#     datastore: Path = Path("$GAMMAPY_DATA/hess-dl3-dr1/")
#     obs_ids: List[int] = []
#     obs_file: Path = None
#     obs_cone: SpatialCircleConfig = SpatialCircleConfig()
#     obs_time: TimeRangeConfig = TimeRangeConfig()
#     required_irf: List[RequiredHDUEnum] = ["aeff", "edisp", "psf", "bkg"]


class LogConfig(GammapyBaseConfig):
    level: str = "info"
    filename: Path = None
    filemode: str = None
    format: str = None
    datefmt: str = None


class GeneralConfig(GammapyBaseConfig):
    log: LogConfig = LogConfig()
    outdir: str = "."
    n_jobs: int = 1
    datasets_file: Path = None
    models_file: Path = None

        

class GeneralCounterpartsConfig(GammapyBaseConfig):
    log: LogConfig = LogConfig()
    outdir: str = "."
    path_file: Path = None

In [ ]:
class DatasetsConfig(GammapyBaseConfig):
    type: ReductionTypeEnum = ReductionTypeEnum.spectrum
    stack: bool = True
    geom: GeomConfig = GeomConfig()
    selection: List[MapSelectionEnum] = MapDatasetMaker.available_selection
    use_region_center: bool = False
    background: BackgroundConfig = BackgroundConfig()
    safe_mask: SafeMaskConfig = SafeMaskConfig()
    on_region: SpatialCircleConfig = SpatialCircleConfig()
    containment_correction: bool = False
    acceptance: int = None
    acceptance_off: int = None
    alpha: float = None
        
        
class SensitivityConfig(GammapyBaseConfig):
    gamma_min: int = None
    n_sigma: int = None
    bkg_syst_fraction: float = None

                
class ObservationsParametersConfig(GammapyBaseConfig):
    livetime: QuantityType = None
    offset: QuantityType = None
    on_region_radius: AngleType = None
    n_obs: int = None
        
        
class ObservationsConfig(GammapyBaseConfig):
    parameters: ObservationsParametersConfig = ObservationsParametersConfig()
    datastore: Path = Path("$GAMMAPY_DATA/hess-dl3-dr1/")
    obs_ids: List[int] = []
    obs_file: Path = None
    obs_cone: SpatialCircleConfig = SpatialCircleConfig()
    obs_time: TimeRangeConfig = TimeRangeConfig()
    required_irf: List[RequiredHDUEnum] = ["aeff", "edisp", "psf", "bkg"]


class LogConfig(GammapyBaseConfig):
    level: str = "info"
    filename: Path = None
    filemode: str = None
    format: str = None
    datefmt: str = None


class GeneralConfig(GammapyBaseConfig):
    log: LogConfig = LogConfig()
    outdir: str = "."
    n_jobs: int = 1
    datasets_file: Path = None
    models_file: Path = None

        
class TargetConfig(GammapyBaseConfig):
    name: str = None
    position: SkyCoordConfig = SkyCoordConfig()
    model:  dict = {}
        
class IrfsConfig(GammapyBaseConfig):
    opt: IrfType = ['South', 'AverageAz', '20deg', '50h']

class SimulationConfig(GammapyBaseConfig):
    """Gammapy analysis configuration."""

    general: GeneralConfig = GeneralConfig()
    target: TargetConfig = TargetConfig()
    observations: ObservationsConfig = ObservationsConfig()
    irfs: IrfsConfig = IrfsConfig()
    sensitivity: SensitivityConfig = SensitivityConfig()
    datasets: DatasetsConfig = DatasetsConfig()
    fit: FitConfig = FitConfig()
    flux_points: FluxPointsConfig = FluxPointsConfig()
    excess_map: ExcessMapConfig = ExcessMapConfig()
    light_curve: LightCurveConfig = LightCurveConfig()

    def __str__(self):
        """Display settings in pretty YAML format."""
        info = self.__class__.__name__ + "\n\n\t"
        data = self.to_yaml()
        data = data.replace("\n", "\n\t")
        info += data
        return info.expandtabs(tabsize=4)

    @classmethod
    def read(cls, path):
        """Reads from YAML file."""
        config = read_yaml(path)
        return SimulationConfig(**config)


    @classmethod
    def from_yaml(cls, config_str):
        """Create from YAML string."""
        settings = yaml.safe_load(config_str)
        return SimulationConfig(**settings)


    def write(self, path, overwrite=False):
        """Write to YAML file."""
        path = make_path(path)
        if path.exists() and not overwrite:
            raise IOError(f"File exists already: {path}")
        path.write_text(self.to_yaml())


    def to_yaml(self):
        """Convert to YAML string."""
        # Here using `dict()` instead of `json()` would be more natural.
        # We should change this once pydantic adds support for custom encoders
        # to `dict()`. See https://github.com/samuelcolvin/pydantic/issues/1043
        config = json.loads(self.json())
        return yaml.dump(
            config, sort_keys=False, indent=4, width=80, default_flow_style=None
        )

    def set_logging(self):
        """Set logging config.

        Calls ``logging.basicConfig``, i.e. adjusts global logging state.
        """
        self.general.log.level = self.general.log.level.upper()
        logging.basicConfig(**self.general.log.dict())
        log.info("Setting logging config: {!r}".format(self.general.log.dict()))


    def update(self, config=None):
        """Update config with provided settings.

        Parameters
        ----------
        config : string dict or `SimulationConfig` object
            Configuration settings provided in dict() syntax.
        """
        if isinstance(config, str):
            other = SimulationConfig.from_yaml(config)
        elif isinstance(config, SimulationConfig):
            other = config
        else:
            raise TypeError(f"Invalid type: {config}")

        config_new = deep_update(
            self.dict(exclude_defaults=True), other.dict(exclude_defaults=True)
        )
        return SimulationConfig(**config_new)


    @staticmethod
    def _get_doc_sections():
        """Returns dict with commented docs from docs file"""
        doc = defaultdict(str)
        with open(DOCS_FILE) as f:
            for line in filter(lambda line: not line.startswith("---"), f):
                line = line.strip("\n")
                if line.startswith("# Section: "):
                    keyword = line.replace("# Section: ", "")
                doc[keyword] += line + "\n"
        return doc

In [ ]:



# # Define sky model to used simulate the data.
# # Here we use a Gaussian spatial model and a Power Law spectral model.
# spatial_model = GaussianSpatialModel(lon_0="0.2 deg", lat_0="0.1 deg", sigma="0.3 deg", frame="galactic")
# spectral_model = ExpCutoffPowerLawSpectralModel(index=2.2, amplitude="1e-12 cm-2 s-1 TeV-1", reference="1 TeV", 
#                                                 lambda_=1/40. * u.Unit("TeV-1"))
# model_simu = SkyModel(
#     spatial_model=spatial_model,
#     spectral_model=spectral_model,
#     name="model-simu",
# )
# bkg_model = FoVBackgroundModel(dataset_name="dataset-simu")

# models = Models([model_simu, bkg_model])



# # In[ ]:


# from config import *

# from astropy import units as u


# from feupy.utils.scripts import pickling, unpickling
# from feupy.plotters import setting_leg_style, show_SED

# from feupy.analysis.config import SimulationConfig
# from feupy.analysis.core import Simulation
# from feupy.cta.irfs import Irfs




# # In[ ]:


# from gammapy.data.observations import Observations


# # In[ ]:


# analysis = Simulation(config)

# config.observations.irfs_config

# config.observations.irfs_config = ['South', 'AverageAz', '20deg', '50h']

# config.observations.livetime = 1*u.h

# pointing = analysis.set_pointing()

# analysis.create_geometry()


# dataset_empty = analysis.create_reference_dataset()



# maker = analysis.create_dataset_maker()
# mask_maker = analysis.create_safe_mask_maker()





# obs = analysis.create_observation()





# # maker.run(dataset_empty, obs)



# # analysis.create_observation()

# # analysis.observations

# # analysis.spectrum_extraction()



# # In[ ]:


# print(dataset_empty)


# # In[ ]:


# datasets = Datasets()
# models = Models()

# datasets_simulation = []
# models_simulation = []

# analysis_counterparts = []
# datasets_counterparts = []

# analysis_all = []

# leg_style = {}


# # In[ ]:


# pulsar_name = PULSARS[1]
# pulsar_name = 'PSR J1826-1256'


# # In[ ]:


# analysis_path = create_path_analysis(pulsar_name)
# # analysis_counterparts = unpickling(analysis_path / "counterparts")
# # analysis_simulation = unpickling(f"{analysis_path}/analysis_all")


# # In[ ]:


# config = SimulationConfig()

# config.general.path_file = analysis_path


# # In[ ]:


# print(config)


# # In[ ]:





# # In[ ]:


# models.extend(analysis_counterparts.datasets.models)


# # In[ ]:


# datasets_counterparts = analysis_counterparts.datasets

# # datasets.extend(datasets_counterparts)


# # In[ ]:


# leg_style = setting_leg_style(leg_style, names=datasets.names)
# leg_style = setting_leg_style(leg_style, names=datasets.models.names)


# # In[ ]:


# # set(opts_selection).issubset(opts)


# # In[ ]:


# target_name = pulsar_name 
# catalog = catalog_pulsar


# # In[ ]:


# target_name = pulsar_name 
# catalog = catalog_pulsar
# source = catalog[target_name]
# name = source.name
# pos_ra = source.position.ra
# pos_dec = source.position.dec
# target = Target(name, pos_ra, pos_dec)


# # In[ ]:


# position = target.position


# # In[ ]:


# on_region_radius=0.15*u.deg


# # In[ ]:


# target.dict


# # In[ ]:


# roi = ROI(target=target, radius=on_region_radius)


# # In[ ]:


# from feupy.utils.coordinates import skcoord_to_dict


# # In[ ]:


# n_obs=10


# # In[ ]:


# pointing_angle = 0*u.deg
# offset=1*u.deg 


# # In[ ]:


# # print(config)


# # In[ ]:


# print(config.datasets.on_region)


# # In[ ]:


# on_region = {'lon':276.53554167*u.deg,
#  'lat': -12.9425 *u.deg,
#  'frame': 'icrs',
#  'radius': 0.15*u.deg,
# }


# # In[ ]:





# # In[ ]:





# # In[ ]:






# # In[ ]:


# config.datasets.on_region = o


# # In[ ]:


# target.dict


# # In[ ]:


# # config.datasets.on_region = target.dict['position']

# config.datasets.on_region = on_region


# # In[ ]:


# config.observations.offset = offset
# config.observations.pointing.angle = pointing_angle


# # In[ ]:


# n_obs = 3


# # In[ ]:


# # config.observations.parameters.on_region_radius = on_region_radius


# # In[ ]:


# # config.observations.parameters.offset = offset

# # config.observations.parameters.on_region_radius = on_region_radius

# config.observations.n_obs = n_obs


# # In[ ]:


# e_edges_min=0.1*u.TeV 
# e_edges_max=100.*u.TeV
# nbin_edges = 5
# name = 'energy'


# # In[ ]:


# config.datasets.geom.axes.energy.min = e_edges_min
# config.datasets.geom.axes.energy.max = e_edges_max
# config.datasets.geom.axes.energy.nbins = nbin_edges
# config.datasets.geom.axes.energy.name = 'energy'


# # In[ ]:


# print(config)


# # In[ ]:





# # In[ ]:





# # In[ ]:





# # In[ ]:


# analysis = Simulation(config)


# # In[ ]:


# analysis.set_pointing()


# # In[ ]:





# # In[ ]:





# # In[ ]:


# analysis.create_geometry()


# # In[ ]:


# analysis.create_reference_dataset()


# # In[ ]:


# print(analysis.config)


# # In[ ]:


# config.observations.irfs_config = irfs_opt
# # config.observations.parameters.offset = offset

# # config.observations.pointing.angle = pointing_angle
# config.observations.livetime = livetime


# # config.observations.target = target.dict
# obs_label = f"{get_obs_label(config.observations)}"


# # In[ ]:


# opts = irfs_opt.copy()
# opts.append(f"livetime:{livetime}".strip())
# opts.append(f"offset: {offset}".strip())
# opts.append(f"on_region_radius:{on_region_radius}".strip())
# opts.append(f"n_obs:{n_obs}".strip())
# config.general.opts = opts


# # In[ ]:


# config.datasets.map_selection = ["edisp", "background", "exposure"]
# config.datasets.safe_mask.methods = ["bkg-peak"]
# config.datasets.safe_mask.parameters = {'aeff_percent': 10}



# # In[ ]:





# # In[ ]:


# config.datasets_onoff.acceptance = 1
# config.datasets_onoff.acceptance_off = 5


# # In[ ]:


# config.statistics.alpha = 0.2
# config.sensitivity.gamma_min = 5 
# config.sensitivity.n_sigma = 3 
# config.sensitivity.bkg_syst_fraction = 0.10



# config.datasets.geom.axes.energy_true.min = .3*e_edges_min
# config.datasets.geom.axes.energy_true.max = 3*e_edges_max
# config.datasets.geom.axes.energy_true.nbins = 8
# config.datasets.geom.axes.energy_true.name = 'energy_true'



# # In[ ]:


# config.fit.fit_range.min = 0.1 * u.TeV
# config.fit.fit_range.max = 100 * u.TeV
# config.fit.required_fit = ["joint", "estacked", "separated"]


# # In[ ]:


# analysis.run_onoff()


# # In[ ]:


# print(f"Simulation for Livetime = {livetime}")

    

    


#     model = model.copy(name=f"model {obs_label}")
#     config.observations.target.model = model.to_dict()


#     config.flux_points.energy.min = e_edges_min
#     config.flux_points.energy.max = e_edges_max
#     config.flux_points.energy.nbins = nbin_edges
#     config.flux_points.source = model.name
#     config.flux_points.parameters = {"selection_optional": "all"}



#     analysis = Simulation(config)
#     analysis.create_spectrum_dataset()
#     analysis.run_onoff()
#     analysis.fit_model_parameters()
#     analysis.fit_joint()
# #     analysis.estimate_flux_points()

#     analysis_all.append(analysis)


# # In[ ]:


# results = []
# for idx, livetime in enumerate(livetime_scan):
#     get_ipython().run_line_magic('time', '')
#     print(f"Simulation for Livetime = {livetime}")
#     config = SimulationConfig()
    
#     config.general.path_file = analysis_path
    
#     config.observations.irfs.opt = irfs_opt
#     config.observations.parameters.offset = offset
#     config.observations.pointing.angle = pointing_angle
    
#     config.observations.parameters.on_region_radius = on_region_radius
#     config.observations.parameters.livetime = livetime
#     config.observations.parameters.n_obs = n_obs
#     config.observations.target = target.dict
#     obs_label = f"{get_obs_label(config.observations)}"
    
#     opts = irfs_opt.copy()
#     opts.append(f"livetime:{livetime}".strip())
#     opts.append(f"offset: {offset}".strip())
#     opts.append(f"on_region_radius:{on_region_radius}".strip())
#     opts.append(f"n_obs:{n_obs}".strip())
#     config.general.opts = opts
    

#     config.datasets.selection = ["edisp", "background", "exposure"]
#     config.datasets.safe_mask.methods = ["bkg-peak"]
#     config.datasets.safe_mask.parameters = {'aeff_percent': 10}

#     config.datasets_onoff.acceptance = 1
#     config.datasets_onoff.acceptance_off = 5

#     config.statistics.alpha = 0.2
#     config.sensitivity.gamma_min = 5 
#     config.sensitivity.n_sigma = 3 
#     config.sensitivity.bkg_syst_fraction = 0.10

#     config.datasets.geom.axes.energy.min = ".1 TeV"
#     config.datasets.geom.axes.energy.max = "100 TeV"
#     config.datasets.geom.axes.energy.nbins = 5
#     config.datasets.geom.axes.energy.name = 'energy'

#     config.datasets.geom.axes.energy_true.min = .3*e_edges_min
#     config.datasets.geom.axes.energy_true.max = 3*e_edges_max
#     config.datasets.geom.axes.energy_true.nbins = 8
#     config.datasets.geom.axes.energy_true.name = 'energy_true'

#     model = model.copy(name=f"model {obs_label}")
#     config.observations.target.model = model.to_dict()

#     config.flux_points.energy.min = e_edges_min
#     config.flux_points.energy.max = e_edges_max
#     config.flux_points.energy.nbins = nbin_edges
#     config.flux_points.source = model.name
#     config.flux_points.parameters = {"selection_optional": "all"}

#     config.fit.fit_range.min = 0.1 * u.TeV
#     config.fit.fit_range.max = 100 * u.TeV
#     config.fit.required_fit = ["joint", "estacked", "separated"]
#     analysis = Simulation(config)
#     analysis.create_spectrum_dataset()
#     analysis.run_onoff()
#     analysis.fit_model_parameters()
#     analysis.fit_joint()
# #     analysis.estimate_flux_points()

#     analysis_all.append(analysis)


# # In[ ]:


# analysis.


# # In[ ]:


# spectrum = datasets_counterparts.models[0].spectral_model


# # In[ ]:


# model = datasets_counterparts.models[0].copy(name="model")
# model_fit = datasets_counterparts.models[0].copy(name="model-fit")


# # In[ ]:


# irfs_opt = ['North', 'AverageAz', '40deg', '50h']
# livetime=20*u.h 


# # In[ ]:





# # In[ ]:


# livetime_scan = [1, 5, 10.0, 20.] * u.hr


# # In[ ]:


# livetime_scan = [1] * u.hr


# # In[ ]:


# # config.write(make_path(f"{analysis_path}/{name_to_txt(pulsar_name)}.yaml"), overwrite=True)


# # In[ ]:


# model = Models([model])
# model = Models.from_dict(models.to_dict())


# # In[ ]:


# model.names


# # In[ ]:





# # In[ ]:


# get_obs_label(analysis.config.observations)


# # In[ ]:


# analysis_all[0].dataset_stacked.info_dict()


# # In[ ]:


# analysis.


# # In[ ]:


# analysis.config.observations.target.model['name'] = "target"


# # In[ ]:


# analysis.config.observations.target.model['name'] 


# # In[ ]:


# type(analysis.config.observations.target.model)


# # In[ ]:


# type(target.sky_model)


# # In[ ]:


# analysis.fit_result.parameters.to_table()


# # In[ ]:


# analysis.run_fit()


# # In[ ]:


# analysis.fit_result.parameters.to_table()


# # In[ ]:


# # pickling(analysis_all, analysis_path / 'analysis_all')


# # In[ ]:





# # In[ ]:





# # In[ ]:




